In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.append('..\..')
import matplotlib
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import pandas as pd
import pickle
from scipy.signal import decimate
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean, plot_one_side, calculate_error_bars
from utils.plotting import get_photometry_around_event
from utils.individual_trial_analysis_utils import ZScoredTraces, SessionData, CueAlignedData, CueAlignedSidedData, ChoiceAlignedData
from set_global_params import processed_data_path, experiment_record_path, reproduce_figures_path, spreadsheet_path
from utils.stats import cohen_d_paired
import os

In [2]:
repro_dir = os.path.join(reproduce_figures_path, 'ED_fig5')
repro_traces_peaks_file = os.path.join(repro_dir, 'silence_traces_and_peaks.pkl')
repro_time_points_file = os.path.join(repro_dir, 'silence_trace_time_points.npy')
if ((os.path.exists(repro_traces_peaks_file)) & (os.path.exists(repro_time_points_file))) :
    print('You have repro data, skip to plotting at end of file')

# Extracting traces and responses (do not run if you have repro data)

In [3]:
def get_all_experimental_records():
    experiment_record = pd.read_csv(experiment_record_path)
    experiment_record['date'] = experiment_record['date'].astype(str)
    return experiment_record

In [4]:
def open_experiment(experiment_to_add):
    for index, experiment in experiment_to_add.iterrows():
        session_traces = SessionData(experiment['fiber_side'], experiment['recording_site'], experiment['mouse_id'], experiment['date'])
    return session_traces

In [5]:
class CustomAlignedData(object):
    def __init__(self, session_data, params):
        saving_folder = processed_data_path + session_data.mouse + '\\'
        restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = session_data.mouse + '_' + session_data.date + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)

        fiber_options = np.array(['left', 'right'])
        fiber_side_numeric = (np.where(fiber_options == session_data.fiber_side)[0] + 1)[0]
        contra_fiber_side_numeric = (np.where(fiber_options != session_data.fiber_side)[0] + 1)[0]

        self.ipsi_data = ZScoredTraces(trial_data, dff, params, fiber_side_numeric, 0)
        self.ipsi_data.get_peaks()
        self.contra_data = ZScoredTraces(trial_data, dff,params, contra_fiber_side_numeric, 0)
        self.contra_data.get_peaks()

In [6]:
repro_dir = os.path.join(reproduce_figures_path,'ED_fig5')
mouse_id = 'SNL_photo70'
date = '20220408'
repro_file = os.path.join(repro_dir, f'silence_behavioural_data_{mouse_id}.csv')
if not os.path.exists(repro_file):
    all_experiments = get_all_experimental_records()
    experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
    session_data = open_experiment(experiment_to_process)
    saving_folder = os.path.join(processed_data_path, mouse_id)
    restructured_data_filename = mouse_id + '_' + date + '_' + 'restructured_data.pkl'
    trial_data_SNL_photo70 = pd.read_pickle(os.path.join(saving_folder, restructured_data_filename))
    trial_data_SNL_photo70.to_csv(repro_file)
else:
    all_experiments = get_all_experimental_records()
    experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
    session_data = open_experiment(experiment_to_process)
    trial_data_SNL_photo70 = pd.read_csv(repro_file)


In [7]:
cot_trials = trial_data_SNL_photo70[trial_data_SNL_photo70['Sound type'] == 0]

In [8]:
silence_trials = trial_data_SNL_photo70[trial_data_SNL_photo70['Sound type'] == 1]

In [9]:
params = {'state_type_of_interest': 5,
    'outcome': 2,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': 0,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
    'cue': None}
aligned_data = CustomAlignedData(session_data, params)

(5095, 18)
(5095, 18)


In [10]:
_, cot_inds, _ = np.intersect1d(aligned_data.contra_data.trial_nums, cot_trials['Trial num'], return_indices=True)
_, silence_inds, _ = np.intersect1d(aligned_data.contra_data.trial_nums, silence_trials['Trial num'], return_indices=True)

In [11]:
peaks = aligned_data.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds  if peaks[trial].size > 0]

In [12]:
all_time_points = decimate(aligned_data.contra_data.time_points, 50) # need to downsample more to keep within csv limits (used to be q=10)
time_window_size = 2
bool_idx = (all_time_points < time_window_size) & (all_time_points >= -time_window_size)
time_points = all_time_points[bool_idx]


In [13]:
cot_traces = aligned_data.contra_data.sorted_traces[cot_inds, :]
silence_traces = aligned_data.contra_data.sorted_traces[silence_inds, :]
decimated_cot_traces = decimate(cot_traces, 50)[:, bool_idx]
decimated_silence_trace = decimate(silence_traces, 50)[:, bool_idx]

# this is the example mouse so we save the csv out with per trial traces too 
spreadsheet_file_path = os.path.join(spreadsheet_path, 'ED_fig5')
if not os.path.exists(os.path.join(spreadsheet_file_path, 'ED_fig5B_silence_traces.csv')):
    df_for_spreadsheet = pd.DataFrame(decimated_silence_trace.T)
    df_for_spreadsheet.insert(0, "Timepoints", time_points)
    df_for_spreadsheet.to_csv(os.path.join(spreadsheet_file_path, 'ED_fig5B_silence_traces.csv'))
if not os.path.exists(os.path.join(spreadsheet_file_path, 'ED_fig5B_tones_traces.csv')):
    df_for_spreadsheet = pd.DataFrame(decimated_cot_traces.T)
    df_for_spreadsheet.insert(0, "Timepoints", time_points)
    df_for_spreadsheet.to_csv(os.path.join(spreadsheet_file_path, 'ED_fig5B_tones_traces.csv'))
    
    
peaks = aligned_data.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds if peaks[trial].size > 0]
mean_cot_trace = np.mean(decimated_cot_traces, axis=0)
cot_error_bar_lower, cot_error_bar_upper = calculate_error_bars(mean_cot_trace,
                                                        decimated_cot_traces,
                                                        error_bar_method='sem')
                 
mean_silence_trace = np.mean(decimated_silence_trace, axis=0)

silence_error_bar_lower, silence_error_bar_upper = calculate_error_bars(mean_silence_trace,
                                                        decimated_silence_trace,
                                                        error_bar_method='sem')

In [14]:
mouse_silence_data = {'mouse': mouse_id, 'tone': [mean_cot_trace], 'tone error bar lower': [cot_error_bar_lower], 'tone error  bar upper': [cot_error_bar_upper], 
                     'silence': [mean_silence_trace], 'silence error bar lower': [silence_error_bar_lower], 'silence error bar upper': [silence_error_bar_upper], 
                     'mean tone peak': np.mean(cot_peaks), 'mean silence peak': np.mean(silence_peaks)}

In [15]:
all_mouse_silence_df = pd.DataFrame(mouse_silence_data)

In [16]:
mouse_id = 'SNL_photo43'
date = '20210603'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)

In [17]:
params = {'state_type_of_interest': 5,
    'outcome': 2,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': 0,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
    'cue': None}
aligned_data = CustomAlignedData(session_data, params)
cot_inds = np.where(aligned_data.contra_data.trial_nums <=149)[0]
silence_inds = np.where(aligned_data.contra_data.trial_nums >149)[0]
cot_traces = aligned_data.contra_data.sorted_traces[cot_inds, :]
silence_traces = aligned_data.contra_data.sorted_traces[silence_inds, :]
decimated_cot_traces = decimate(cot_traces, 50)[:, bool_idx]
decimated_silence_trace = decimate(silence_traces, 50)[:, bool_idx]
peaks = aligned_data.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds if peaks[trial].size > 0]
mean_cot_trace = np.mean(decimated_cot_traces, axis=0)
cot_error_bar_lower, cot_error_bar_upper = calculate_error_bars(mean_cot_trace,
                                                        decimated_cot_traces,
                                                        error_bar_method='sem')
                 
mean_silence_trace = np.mean(decimated_silence_trace, axis=0)

silence_error_bar_lower, silence_error_bar_upper = calculate_error_bars(mean_silence_trace,
                                                        decimated_silence_trace,
                                                        error_bar_method='sem')
mouse_silence_data = {'mouse': mouse_id, 'tone': [mean_cot_trace], 'tone error bar lower': [cot_error_bar_lower], 'tone error  bar upper': [cot_error_bar_upper], 
                     'silence': [mean_silence_trace], 'silence error bar lower': [silence_error_bar_lower], 'silence error bar upper': [silence_error_bar_upper], 
                     'mean tone peak': np.mean(cot_peaks), 'mean silence peak': np.mean(silence_peaks)}

(5730, 17)
(5730, 17)


In [18]:
mouse_silence_df = pd.DataFrame(mouse_silence_data)

In [19]:
all_mouse_silence_df = pd.concat([all_mouse_silence_df, mouse_silence_df])

In [20]:
mouse_id = 'SNL_photo37'
date = '20210603'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)

In [21]:
params = {'state_type_of_interest': 5,
    'outcome': 2,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': 0,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
    'cue': None}
aligned_data = CustomAlignedData(session_data, params)

cot_inds = np.where(aligned_data.contra_data.trial_nums <=149)[0]
silence_inds = np.where(aligned_data.contra_data.trial_nums >149)[0]
peaks = aligned_data.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds  if peaks[trial].size > 0]

cot_traces = aligned_data.contra_data.sorted_traces[cot_inds, :]
silence_traces = aligned_data.contra_data.sorted_traces[silence_inds, :]
decimated_cot_traces = decimate(cot_traces, 50)[:, bool_idx]
decimated_silence_trace = decimate(silence_traces, 50)[:, bool_idx]
peaks = aligned_data.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds if peaks[trial].size > 0]
mean_cot_trace = np.mean(decimated_cot_traces, axis=0)
cot_error_bar_lower, cot_error_bar_upper = calculate_error_bars(mean_cot_trace,
                                                        decimated_cot_traces,
                                                        error_bar_method='sem')
                 
mean_silence_trace = np.mean(decimated_silence_trace, axis=0)

silence_error_bar_lower, silence_error_bar_upper = calculate_error_bars(mean_silence_trace,
                                                        decimated_silence_trace,
                                                        error_bar_method='sem')
mouse_silence_data = {'mouse': mouse_id, 'tone': [mean_cot_trace], 'tone error bar lower': [cot_error_bar_lower], 'tone error  bar upper': [cot_error_bar_upper], 
                     'silence': [mean_silence_trace], 'silence error bar lower': [silence_error_bar_lower], 'silence error bar upper': [silence_error_bar_upper], 
                     'mean tone peak': np.mean(cot_peaks), 'mean silence peak': np.mean(silence_peaks)}

(5178, 17)
(5178, 17)


In [22]:
mouse_silence_df = pd.DataFrame(mouse_silence_data)

In [23]:
all_mouse_silence_df = pd.concat([all_mouse_silence_df, mouse_silence_df])

In [24]:
all_mouse_silence_df = all_mouse_silence_df.reset_index(drop=True)

In [25]:
time_points.shape

(807,)

In [26]:
repro_traces_peaks_file = os.path.join(repro_dir, 'silence_traces_and_peaks.pkl')
repro_time_points_file = os.path.join(repro_dir, 'silence_trace_time_points.npy')
if not os.path.exists(repro_traces_peaks_file):
    with open(repro_traces_peaks_file, 'wb') as f:
        pickle.dump(all_mouse_silence_df, f)
if not os.path.exists(repro_time_points_file):
    np.save(repro_time_points_file, time_points)

# Just Plotting

Average accross mice traces

In [27]:
repro_traces_peaks_file = os.path.join(repro_dir, 'silence_traces_and_peaks.pkl')
repro_time_points_file = os.path.join(repro_dir, 'silence_trace_time_points.npy')
all_mouse_silence_data = pd.read_pickle(repro_traces_peaks_file)
time_points = np.load(repro_time_points_file)

In [28]:
concatenated_tone_traces = np.array([all_mouse_silence_df['tone'].values[i] for i in all_mouse_silence_df.index.values])

In [29]:
spreadsheet_file_path = os.path.join(spreadsheet_path, 'ED_fig5')
if not os.path.exists(os.path.join(spreadsheet_file_path, 'ED_fig5C_tone_traces.csv')):
    df_for_spreadsheet = pd.DataFrame(concatenated_tone_traces.T)
    df_for_spreadsheet.insert(0, "Timepoints", time_points)
    df_for_spreadsheet.to_csv(os.path.join(spreadsheet_file_path, 'ED_fig5C_tone_traces.csv'))

In [30]:
mean_cot_traces = np.mean(concatenated_tone_traces, axis=0)

In [31]:
concatenated_silence_traces = np.array([all_mouse_silence_df['silence'].values[i] for i in all_mouse_silence_df.index.values])
mean_silence_traces = np.mean(concatenated_silence_traces, axis=0)

In [32]:
spreadsheet_file_path = os.path.join(spreadsheet_path, 'ED_fig5')
if not os.path.exists(os.path.join(spreadsheet_file_path, 'ED_fig5C_silence_traces.csv')):
    df_for_spreadsheet = pd.DataFrame(concatenated_silence_traces.T)
    df_for_spreadsheet.insert(0, "Timepoints", time_points)
    df_for_spreadsheet.to_csv(os.path.join(spreadsheet_file_path, 'ED_fig5C_silence_traces.csv'))

In [33]:
all_mice_silence_error_bar_lower, all_mice_silence_error_bar_upper = calculate_error_bars(mean_silence_traces,
                                                        concatenated_silence_traces,
                                                        error_bar_method='sem')
all_mice_cot_error_bar_lower, all_mice_cot_error_bar_upper = calculate_error_bars(mean_cot_traces,
                                                        concatenated_tone_traces,
                                                        error_bar_method='sem')               

In [34]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']

fig, axs1 = plt.subplots(1,1, figsize=[2.5,2])
colours = cm.viridis(np.linspace(0.2, 0.7, 2))
axs1.plot(time_points, mean_cot_traces, label='normal cue', color=colours[0])
axs1.fill_between(time_points, all_mice_cot_error_bar_lower, all_mice_cot_error_bar_upper, alpha=0.4, linewidth=0, color=colours[0])
axs1.plot(time_points, mean_silence_traces, label='silence', color=colours[1])
axs1.fill_between(time_points, all_mice_silence_error_bar_lower, all_mice_silence_error_bar_upper, alpha=0.4, linewidth=0, color=colours[1])
axs1.set_xlim([-2, 2])
axs1.axvline([0], color='k')
axs1.set_ylabel('z-scored fluorescence', fontsize=8)
axs1.set_xlabel('Time (s)', fontsize=8)

axs1.spines['right'].set_visible(False)
axs1.spines['top'].set_visible(False)
axs1.legend(loc='lower left', bbox_to_anchor=(0.7, 0.8),
            borderaxespad=0, frameon=False,prop={'size': 6 })
plt.tight_layout()


In [63]:
cot_peaks = all_mouse_silence_df['mean tone peak'].values

In [64]:
silence_peaks = all_mouse_silence_df['mean silence peak'].values

In [65]:
ttest_rel(cot_peaks, silence_peaks)

Ttest_relResult(statistic=1.3609088145280699, pvalue=0.3066032786828079)

In [66]:
_ = cohen_d_paired(cot_peaks, silence_peaks)

cohen d:  0.7857210704103156


In [67]:
df_for_plot = all_mouse_silence_df[['mouse', 'mean tone peak', 'mean silence peak']].set_index('mouse').T

In [71]:
peaks_silence_cot_spreadsheet = os.path.join(spreadsheet_file_path, 'ED_fig5D_silence_tones_peaks.csv')
if not os.path.exists(peaks_silence_cot_spreadsheet):
    df_for_plot.to_csv(peaks_silence_cot_spreadsheet)

In [68]:
def comparison_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax,  legend=False, color='grey')
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color, alpha=0.7)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine 
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


In [69]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']


fig, ax = plt.subplots(figsize=[2,2])
comparison_plot(ax, df_for_plot, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.xticks([0, 1], ['tone', 'silence'], fontsize=8)
plt.ylabel('Z-scored fluorescence', fontsize=8)
#ax.text(0.9, 1.9, 'p-value = {0:.3f}'.format(pval))
ax.set_xlabel(' ')

#significance stars
y = df_for_plot.to_numpy().max() + .2
h = .1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
ax.text(.5, y+h, 'n.s.', ha='center', fontsize=10)

plt.tight_layout()

In [72]:
example_mouse_data = all_mouse_silence_df[all_mouse_silence_df['mouse']== 'SNL_photo70']

In [73]:
example_tone_mean_trace = example_mouse_data['tone'][0]
example_tone_upper_error = example_mouse_data['tone error  bar upper'][0]
example_tone_lower_error = example_mouse_data['tone error bar lower'][0]

In [74]:
example_silence_mean_trace = example_mouse_data['silence'][0]
example_silence_upper_error = example_mouse_data['silence error bar upper'][0]
example_silence_lower_error = example_mouse_data['silence error bar lower'][0]

In [271]:
font = {'size': 7}
matplotlib.rc('font', **font)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.sans-serif'] = 'Arial'
matplotlib.rcParams['font.family']

fig, axs1 = plt.subplots(1,1, figsize=[2.5,2])
colours = cm.viridis(np.linspace(0.2, 0.7, 2))
axs1.plot(time_points, example_tone_mean_trace, label='normal cue', color=colours[0])
axs1.fill_between(time_points, example_tone_lower_error, example_tone_upper_error, alpha=0.4, linewidth=0, color=colours[0])
axs1.plot(time_points, example_silence_mean_trace, label='silence', color=colours[1])
axs1.fill_between(time_points, example_silence_lower_error, example_silence_upper_error, alpha=0.4, linewidth=0, color=colours[1])
axs1.set_xlim([-2,2])
axs1.axvline([0], color='k')
axs1.set_ylabel('z-scored fluorescence', fontsize=8)
axs1.set_xlabel('Time (s)', fontsize=8)

axs1.spines['right'].set_visible(False)
axs1.spines['top'].set_visible(False)
axs1.legend(loc='lower left', bbox_to_anchor=(0.7, 0.8),
            borderaxespad=0, frameon=False,prop={'size': 6 })
plt.tight_layout()